In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

def load_data():
    return pd.read_csv('.data/train.En.csv', usecols=['tweet', 'sarcastic']).dropna()


2021-12-14 19:17:44.033542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-14 19:17:44.033588: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Load Data

In [30]:
dataset = load_data()

sentences = dataset['tweet'].values
labels = dataset['sarcastic'].values

X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.1, random_state=42, stratify=labels)

# Tokenize

In [31]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',disable_tqdm=True)

In [32]:
# attention mask helps to guide the model and know if there was missing value

def tokenize(X, y):
  input_ids, attention_masks, outputs = [], [], []
  for sentence, label in zip(X, y):
    sentence_tokens = bert_tokenizer.encode_plus(sentence, max_length=10, pad_to_max_length=True, truncation=True)
    input_ids.append(sentence_tokens["input_ids"])
    attention_masks.append(sentence_tokens['attention_mask'])
    outputs.append(label)

  return input_ids, attention_masks, outputs

train_input_ids, train_attention_masks, train_outputs = tokenize(X_train, y_train)
test_input_ids, test_attention_masks, test_outputs = tokenize(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# Create Model

In [33]:
bert_classification_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = len(set(labels)))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
bert_classification_model.compile(optimizer=Adam(learning_rate=0.01),
                                  loss=binary_crossentropy,
                                  metrics=['accuracy'])

bert_classification_model.fit([np.array(train_input_ids), np.array(train_attention_masks)], np.array(train_outputs), epochs=15, verbose=2)

Epoch 1/15


KeyboardInterrupt: ignored

In [ ]:
bert_classification_model.evaluate([np.array(test_input_ids), np.array(test_attention_masks)],
                                    np.array(test_outputs))